In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import torch
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

In [ ]:
from pathlib import Path
from research.imagenet_classes import classes

dataset_path = Path("G:\\Github Repositories\\exploiting-gan-internal-capacity\\inverses\\dataset.hdf5")
dataset = h5py.File(dataset_path, "r")

N = dataset['xtrain'].shape[0]
@interact(i=(0, N-1))
def show(i):
    img = dataset['xtrain'][i]
    img = np.moveaxis(img, 0, -1)
    
    label_id = dataset['ytrain'][i]
    
    print(classes[label_id])
    
    plt.imshow(img)

In [37]:
from research.things_dataset import ThingsDataset
import torchvision.transforms as T

transform = T.Compose([
    T.Resize(128),
    T.ToTensor()
])

things_dataset = ThingsDataset(
    root="X:\\Datasets\\EEG\\Things-concepts-and-images\\",
    transform=transform
)
things_dataset.image_concept_ids

#print(things_dataset.image_concept_ids)

@interact(i=(0, len(things_dataset)-1))
def show(i):
    image = things_dataset[i]
    print(image['unique_id'])
    data = image['data']
    data = torch.moveaxis(data, 0, -1)
    
    plt.imshow(data)
    plt.show()

interactive(children=(IntSlider(value=13053, description='i', max=26106), Output()), _dom_classes=('widget-int…

In [3]:
torch.cuda.is_available()

True

In [4]:
from dall_e import map_pixels, unmap_pixels, load_model

device = torch.device('cuda')

# For faster load times, download these files locally and use the local paths instead.
enc = load_model("X:\Models\dall-e\encoder.pkl", device)
dec = load_model("X:\Models\dall-e\decoder.pkl", device)

In [14]:
import torch.nn.functional as F
from IPython.display import display, display_markdown

@interact(i=(0, len(things_dataset)-1))
def show(i):
    image = things_dataset[i]
    print(image['concept_name'])
    data = image['data']
    data = map_pixels(data[None])
    data = data.to(device)
    
    z_logits = enc(data)
    print(z_logits.shape)
    
    z = torch.argmax(z_logits, axis=1)
    z = F.one_hot(z, num_classes=enc.vocab_size).permute(0, 3, 1, 2).float()
    print(z.shape)

    x_stats = dec(z).float()
    x_rec = unmap_pixels(torch.sigmoid(x_stats[:, :3]))
    x_rec = T.ToPILImage(mode='RGB')(x_rec[0])

    display_markdown('Reconstructed image:')
    display(x_rec)

interactive(children=(IntSlider(value=13053, description='i', max=26106), Output()), _dom_classes=('widget-int…

In [16]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [56]:
from nltk.corpus import wordnet as wn
from research.imagenet_wordnet_ids import imagenet_wordnet_ids

things_wordnet_ids = list(things_dataset.concepts['Wordnet ID4'])

print(imagenet_wordnet_ids)

imagenet_synsets = [wn.synset(id) for id in imagenet_wordnet_ids]
things_synsets = [wn.synset(id) for id in things_wordnet_ids]

imagenet_things_similarity = [
    [imagenet_synset.path_similarity(things_synset) for imagenet_synset in imagenet_synsets]
    for things_synset in things_synsets
]

['tench.n.01', 'goldfish.n.01', 'great_white_shark.n.01', 'tiger_shark.n.01', 'hammerhead.n.01', 'electric_ray.n.01', 'stingray.n.01', 'cock.n.01', 'hen.n.01', 'ostrich.n.01', 'brambling.n.01', 'goldfinch.n.01', 'house_finch.n.01', 'junco.n.01', 'indigo_bunting.n.01', 'robin.n.01', 'bulbul.n.01', 'jay.n.01', 'magpie.n.01', 'chickadee.n.01', 'water_ouzel.n.01', 'kite.n.01', 'bald_eagle.n.01', 'vulture.n.01', 'great_grey_owl.n.01', 'European_fire_salamander.n.01', 'common_newt.n.01', 'eft.n.01', 'spotted_salamander.n.01', 'axolotl.n.01', 'bullfrog.n.01', 'tree_frog.n.01', 'tailed_frog.n.01', 'loggerhead.n.01', 'leatherback_turtle.n.01', 'mud_turtle.n.01', 'terrapin.n.01', 'box_turtle.n.01', 'banded_gecko.n.01', 'common_iguana.n.01', 'American_chameleon.n.01', 'whiptail.n.01', 'agama.n.01', 'frilled_lizard.n.01', 'alligator_lizard.n.01', 'Gila_monster.n.01', 'green_lizard.n.01', 'African_chameleon.n.01', 'Komodo_dragon.n.01', 'African_crocodile.n.01', 'American_alligator.n.01', 'tricerato

WordNetError: lemma 'white_stork' with part of speech 'n' has only 1 sense

In [26]:
wn.synset("john.n.02")

Synset('john.n.02')